<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-02-rnn-lot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lot


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

# 마지막 회차번호

In [ ]:
# 마지막 회차 번호
noEnd ='970' 

# url
# url = 'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={}'.format(noEnd)

url = f'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={noEnd}'

# user-agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [ ]:
result = requests.get(url, headers={"user-agent":user_agent})

In [ ]:
result

In [ ]:
f = BytesIO(result.content)
# df = pd.read_excel(f,sheet_name='excel',skiprows=3)
html_result = pd.read_html(f)

In [ ]:
df_raw = pd.DataFrame(html_result[1])
# df = df.drop([0])
df_raw.head()

In [ ]:
# columns = [col for i, col in enumerate(df.iloc[0])]
# df.columns = columns
# columns

# column name 설정
col_names = [fn+sn if fn != sn else fn for fn, sn in zip(df_raw.iloc[0], df_raw.iloc[1])]  
df_raw.columns = col_names

# 0, 1행 삭제
df_lot = df_raw.drop([0,1], axis=0)

# 컬럼 형변환
df_lot = df_lot.astype({'회차':'uint', '당첨번호1':'uint', '당첨번호2':'uint', '당첨번호3':'uint', '당첨번호4':'uint', '당첨번호5':'uint', '당첨번호6':'uint', '당첨번호보너스':'uint'}, copy=True)

# 당첨금액 형변환
df_lot['1등당첨금액'] = df_lot['1등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['2등당첨금액'] = df_lot['2등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['3등당첨금액'] = df_lot['3등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['4등당첨금액'] = df_lot['4등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['5등당첨금액'] = df_lot['5등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')

# 회차기준 오름차준 정렬
df_lot = df_lot.sort_values('회차', ascending=True)

# 회차필드 index로 설정
df_lot.set_index('회차', inplace=True)

# 필요없는 컬럼 삭제
df_lot.drop(['년도', '추첨일', '1등당첨자수', '2등당첨자수', '3등당첨자수', '4등당첨자수', '5등당첨자수'], axis=1, inplace=True)

df_lot.head(5)

In [ ]:
df_lot.info()

In [ ]:
import numpy as np

# 당첨번호를 원핫인코딩벡터로 변환
def lotto2onehot(numbers):
    onehot = np.zeros(45)
    for number in numbers:
        onehot[int(number)-1] = 1 #로또번호에 해당하는 인덱스에 1로 설정 
    return onehot

# 원핫인코딩벡터를 당첨번호로 변환
def onehot2lotto(onehots):
    numbers = [i+1 for i, val in enumerate(onehots) if val == 1]
    return numbers

In [ ]:
# 함수 동작 확인

lotto = [1,3,5,7,9,11]
onehot_lotto = lotto2onehot(lotto)
print(onehot_lotto)
onehot2lotto(onehot_lotto)

In [ ]:
# 당첨번호 추출
bonus_number = df_lot['당첨번호보너스'].to_numpy()
lotto_numbers = df_lot[['당첨번호1','당첨번호2','당첨번호3', '당첨번호4', '당첨번호5', '당첨번호6']].to_numpy()
lotto_numbers

In [ ]:
# 당첨번호 onehot 인코딩
onehot_lotto_numbers = list(map(lotto2onehot, lotto_numbers))

row_count = len(onehot_lotto_numbers)

x_samples = onehot_lotto_numbers[0:row_count-1]
y_samples = onehot_lotto_numbers[1:row_count]

In [ ]:
np.array(onehot_lotto_numbers).shape

In [ ]:
# train, valid, test set으로 분리
train_idx = (0, 800)
valid_idx = (801, 900)
test_idx = (901, len(onehot_lotto_numbers))

print(f'train: {train_idx}, valid: {valid_idx}, test: {test_idx}')

In [ ]:
# LSTM 모델 생성

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

# 모델을 정의합니다.
model = keras.Sequential([
    keras.layers.LSTM(128, batch_input_shape=(1, 1, 45), return_sequences=False, stateful=True),
    keras.layers.Dense(45, activation='sigmoid')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 매 에포크마다 훈련과 검증의 손실 및 정확도를 기록하기 위한 변수
train_loss = []
train_acc = []
val_loss = []
val_acc = []

# 최대 100번 에포크까지 수행
for epoch in range(100):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    batch_train_loss = []
    batch_train_acc = []
    
    for i in range(train_idx[0], train_idx[1]):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    batch_val_loss = []
    batch_val_acc = []

    for i in range(valid_idx[0], valid_idx[1]):

        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.test_on_batch(xs, ys) #배치만큼 모델에 입력하여 나온 답을 정답과 비교함
        
        batch_val_loss.append(loss)
        batch_val_acc.append(acc)

    val_loss.append(np.mean(batch_val_loss))
    val_acc.append(np.mean(batch_val_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f} val acc {3:0.3f} loss {4:0.3f}'.format(epoch, np.mean(batch_train_acc), 
                                                                                                np.mean(batch_train_loss), 
                                                                                                np.mean(batch_val_acc), 
                                                                                                np.mean(batch_val_loss)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(train_loss, 'y', label='train loss')
loss_ax.plot(val_loss, 'r', label='val loss')

acc_ax.plot(train_acc, 'b', label='train acc')
acc_ax.plot(val_acc, 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
# 88회부터 현재까지 1등부터 5등까지 상금 평균
mean_prize = [df_lot['1등당첨금액'].iloc[87:].mean(), 
              df_lot['2등당첨금액'].iloc[87:].mean(), 
              df_lot['3등당첨금액'].iloc[87:].mean(), 
              df_lot['4등당첨금액'].iloc[87:].mean(), 
              df_lot['5등당첨금액'].iloc[87:].mean()]
print(mean_prize)

In [ ]:
# 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls

In [ ]:
train_total_reward = []
train_total_grade = np.zeros(6, dtype=int)

val_total_reward = []
val_total_grade = np.zeros(6, dtype=int)

test_total_reward = []
test_total_grade = np.zeros(6, dtype=int)

model.reset_states()

print('[No. ] 1st 2nd 3rd 4th 5th 6th Rewards')

for i in range(len(x_samples)):
    xs = x_samples[i].reshape(1, 1, 45)
    ys_pred = model.predict_on_batch(xs) # 모델의 출력값을 얻음
    
    sum_reward = 0
    sum_grade = np.zeros(6, dtype=int) # 6등까지 변수

    for n in range(10): # 10판 수행
        numbers = gen_numbers_from_probability(ys_pred[0])
        
        #i회차 입력 후 나온 출력을 i+1회차와 비교함
        grade, reward = calc_reward(lotto_numbers[i+1], bonus_number[i+1], numbers) 
        
        sum_reward += reward
        sum_grade[grade] += 1

        if i >= train_idx[0] and i < train_idx[1]:
            train_total_grade[grade] += 1
        elif i >= valid_idx[0] and i < valid_idx[1]:
            val_total_grade[grade] += 1
        elif i >= test_idx[0] and i < test_idx[1]:
            val_total_grade[grade] += 1
    
    if i >= train_idx[0] and i < train_idx[1]:
        train_total_reward.append(sum_reward)
    elif i >= valid_idx[0] and i < valid_idx[1]:
        val_total_reward.append(sum_reward)
    elif i >= test_idx[0] and i < test_idx[1]:
        test_total_reward.append(sum_reward)
                        
    print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d} {7:15,d}'.format(i+1, sum_grade[0], sum_grade[1], sum_grade[2], sum_grade[3], sum_grade[4], sum_grade[5], int(sum_reward)))


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

total_reward = train_total_reward + val_total_reward + test_total_reward

plt.plot(total_reward)
plt.ylabel('rewards')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

ax = plt.figure().gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

rewards = [sum(train_total_reward), sum(val_total_reward), sum(test_total_reward)]

class_color=['green', 'blue', 'red']

plt.bar(['train', 'val', 'test'], rewards, color=class_color)
plt.ylabel('rewards')
plt.show()

In [ ]:
# 최대 100번 에포크까지 수행
for epoch in range(100):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    for i in range(len(x_samples)):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))

In [ ]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

for n in range(10):
    numbers = gen_numbers_from_probability(ys_pred[0])
    numbers.sort()
    print('{0} : {1}'.format(n, numbers))
    list_numbers.append(numbers)